In [1]:
import keras
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import CSVLogger, ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.utils.vis_utils import plot_model
from keras.applications.mobilenet_v2 import decode_predictions
from keras.layers.normalization import BatchNormalization

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import os

import tensorflow
from time import time

import pandas

import pickle as pk

from keras.wrappers.scikit_learn import KerasClassifier

import numpy as np

seed = 7

Using TensorFlow backend.


## train all nets


In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15504574288862770091
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11280557671
locality {
  bus_id: 1
  links {
  }
}
incarnation: 13008700569799606219
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:1e.0, compute capability: 3.7"
]


In [3]:
dir(keras.applications.mobilenet_v2)

['MobileNetV2',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'absolute_import',
 'decode_predictions',
 'division',
 'keras_modules_injection',
 'mobilenet_v2',
 'preprocess_input',
 'print_function']

In [4]:
np.random.seed(seed)

In [5]:
data_dir = '/data/oxford102/train'
train_data_dir = "/data/oxford102/train/"
validation_data_dir = "/data/oxford102/train/"
img_width, img_height = 256, 256
batch_size = 128
epochs = 100
nr_categories = 102
nb_train_samples = 4604
nb_validation_samples = 1094 

In [6]:
def get_image_generator(input_processor, img_aug=False):
    if not img_aug:
        train_val_datagen = ImageDataGenerator(preprocessing_function=input_processor, 
                                           validation_split=0.2)
    else: 
        train_val_datagen = ImageDataGenerator(
            rotation_range=40,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            fill_mode='nearest',
            preprocessing_function=input_processor,
            validation_split=0.2)
        
    return train_val_datagen

In [7]:
def get_generators(batch_size, image_size, input_processor, img_aug=False):

    img_width, img_height = image_size
    
    train_val_datagen = get_image_generator(input_processor, img_aug)

    train_generator = train_val_datagen.flow_from_directory(
            train_data_dir,  # this is the target directory
            target_size=(img_width, img_height),  # all images will be resized to 250x250
            batch_size=batch_size,
            subset="training",
            class_mode='categorical')

    validation_generator = train_val_datagen.flow_from_directory(
            train_data_dir,  # this is the target directory
            target_size=(img_width, img_height),  # all images will be resized to 250x250
            subset="validation",
            batch_size=batch_size,
            class_mode='categorical')
    
    return train_generator, validation_generator


In [8]:
#from keras import regularizers

def get_model(network_name="inception_resnet_v2", image_size=(256, 256), verbose=False):
    k.set_learning_phase(0)

    img_width, img_height = image_size
    if network_name == "vgg16":
        base_model = keras.applications.vgg16.VGG16(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))
        input_processor = applications.vgg16.preprocess_input
    elif network_name == "vgg19":
        base_model = keras.applications.vgg19.VGG19(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))
        input_processor = applications.vgg19.preprocess_input
    elif network_name == "inception_resnet_v2":
        base_model = keras.applications.inception_resnet_v2.InceptionResNetV2(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))
        input_processor = applications.inception_resnet_v2.preprocess_input
    elif network_name == "mobilenet_v2":
        base_model = keras.applications.mobilenet_v2.MobileNetV2(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))
        input_processor = applications.mobilenet_v2.preprocess_input
    elif network_name == "xception":
        base_model = keras.applications.xception.Xception(include_top=False, weights='imagenet', input_shape=(img_width, img_height, 3))
        input_processor = applications.xception.preprocess_input
    elif network_name == "resnet50":
        base_model = keras.applications.resnet50.ResNet50(include_top=False, weights='imagenet', input_shape=(img_width, img_height, 3))
        input_processor = applications.resnet50.preprocess_input    
    elif network_name == "inception_v3":
        base_model = keras.applications.inception_v3.InceptionV3(include_top=False, weights='imagenet', input_shape=(img_width, img_height, 3))
        input_processor = applications.inception_v3.preprocess_input
    elif network_name == "mobilenet":
        base_model = keras.applications.mobilenet.MobileNet(include_top=False, weights='imagenet', input_shape=(img_width, img_height, 3))
        input_processor = applications.mobilenet.preprocess_input
    elif network_name == "densenet":
        base_model = keras.applications.densenet.DenseNet201(include_top=False, weights='imagenet', input_shape=(img_width, img_height, 3))
        input_processor = applications.densenet.preprocess_input        
    elif network_name == "nasnet":
        base_model = keras.applications.nasnet.NASNetMobile(include_top=False, weights='imagenet', input_shape=(img_width, img_height, 3))
        input_processor = applications.nasnet.preprocess_input
    else:
        raise Exception("check your network name")

    for layer in base_model.layers[:]:
        layer.trainable = False

        #Adding custom Layers 
    k.set_learning_phase(1)
    x = base_model.output
    x = Flatten()(x)
    x = Dense(1024, activation="relu", 
              #kernel_regularizer=regularizers.l2(0.01),
             #       activity_regularizer=regularizers.l1(0.001)
             )(x)
    x = Dropout(0.5)(x)
    x = BatchNormalization()(x, training=True)
    #x = Dense(102, activation="relu")(x)
    predictions = Dense(nr_categories, activation="softmax")(x)

    _model = Model(input = base_model.input, output = predictions)
    if verbose:
        _model.summary()
    return _model, input_processor

In [9]:
def train_model(params, _model, generators):
    model_name = params["network_name"]
    num_train_img = 4604
    num_val_img = 1094

    np.random.seed(seed)
    log_time = time()
    params['log_time'] = log_time
    batch_size = params.get("batch_size")

    train_generator, validation_generator = generators
    _model.compile(loss = "categorical_crossentropy", optimizer = params["optimizer"], metrics=["accuracy"])


    base = '/data/oxford102/experiments'
    path = os.path.join(base, str(log_time))
    checkpoint = ModelCheckpoint(os.path.join(path, "{}_{}.h5".format(model_name, log_time)), monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
    early = EarlyStopping(monitor='val_acc', min_delta=0, patience=3, verbose=1, mode='auto')
    tensorboard = TensorBoard(log_dir="logs/{}".format(log_time), histogram_freq=0, write_graph=True, write_images=True)
    csv_logger = CSVLogger(os.path.join(path, "{}_{}.csv".format(model_name, log_time)), append=True, separator=';')

    try:
        if not os.path.exists(path):
            os.makedirs(path)
        history_callback = _model.fit_generator(
                train_generator,
                steps_per_epoch=num_train_img // params["batch_size"] // 2,
                epochs=params["epochs"],
                validation_data=validation_generator,
                validation_steps=num_val_img // params["batch_size"],
                callbacks = [checkpoint, early, tensorboard, csv_logger])
    except Exception as e:
        raise(e)
    finally:
        params.pop("optimizer")
        pk.dump(params, open("experimental_params/experiments_{}.pk".format(log_time), "wb"), protocol=pk.HIGHEST_PROTOCOL)
        _model.save_weights(os.path.join(path, 'model_{}_weights_final_{}.h5'.format(model_name, log_time)))  # always save your weights after training or during training
        print(params)
        params
        
    return history_callback

In [10]:
# Hyperparameter optimization

In [22]:
sgd =  optimizers.SGD(lr=0.01, momentum=0.9, decay=1e-6, nesterov=False)

In [23]:
rmsprop = optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)

In [24]:
adagrad = optimizers.Adagrad(lr=0.01, epsilon=None, decay=0.0)

In [25]:
adadelta = optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0)

In [26]:
adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

In [27]:
adamamx = optimizers.Adamax(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0)

In [28]:
nadam = optimizers.Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)

In [18]:
nets = ["mobilenet_v2", "densenet", "nasnet",  "vgg16", "vgg19", "inception_resnet_v2", "mobilenet", "resnet50", "inception_v3"]

In [19]:
params = {'network_name': None,
         'image_aug': False,
         'optimizer_name': False, 
         'optimizer': None,
         'optimizer_params': None, 
         'batch_size': 128,
         'epochs': 100,
         'image_size': (224, 224),
         'log_time': None}

# Train all nets with rmsprop

In [20]:
params['optimizer'] = rmsprop
params['optimizer'] = "rmsprop"

for net in nets:
    print("net:", net)
    params['network_name'] = net
    _model, input_processor = get_model(params["network_name"], image_size=params["image_size"])
    train_generator, validation_generator = get_generators( params["batch_size"], params["image_size"], input_processor)
history = train_model(params, _model, (train_generator, validation_generator))
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

net: mobilenet_v2


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:56: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
17/17 [==============================] - 18s 1s/step - loss: 2.4331 - acc: 0.4752 - val_loss: 1.4218 - val_acc: 0.6689

Epoch 00001: val_acc improved from -inf to 0.66895, saving model to /data/oxford102/experiments/1546837399.7954097/mobilenet_v2_1546837399.7954097.h5
Epoch 2/100
17/17 [==============================] - 9s 556ms/step - loss: 1.1633 - acc: 0.7353 - val_loss: 0.9945 - val_acc: 0.7785

Epoch 00002: val_acc improved from 0.66895 to 0.77847, saving model to /data/oxford102/experiments/1546837399.7954097/mobilenet_v2_1546837399.7954097.h5
Epoch 3/100
17/17 [==============================] - 13s 764ms/step - loss: 0.3068 - acc: 0.9528 - val_loss: 0.7644 - val_acc: 0.8230

Epoch 00003: val_acc improved from 0.77847 to 0.82298, saving model to /data/oxford102/experiments/1546837399.7954097/mobilenet_v2_1546837399.7954097.h5
Epoch 4/100
17/17 [==============================] - 14

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:56: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.


KeyError: 'optimizer'